# Laboratorio: Análisis visual interactivo con Folium
La tasa de éxito de un lanzamiento puede depender de muchos factores, como la masa de la carga útil, el tipo de órbita, etc. También puede depender de la ubicación y la proximidad de un sitio de lanzamiento, es decir, la posición inicial de las trayectorias de los cohetes. Encontrar una ubicación óptima para construir un sitio de lanzamiento implica sin duda muchos factores y, con suerte, podremos descubrir algunos de ellos analizando las ubicaciones de los sitios de lanzamiento existentes.

En los laboratorios de análisis de datos exploratorios anteriores, visualizaste el conjunto de datos de lanzamiento de SpaceX utilizando matplotlib y seaborn y descubriste algunas correlaciones preliminares entre el sitio de lanzamiento y las tasas de éxito. En este laboratorio, realizarás más análisis visuales interactivos utilizando Folium.

## Este laboratorio contiene las siguientes tareas:

* **TAREA 1:** Marcar todos los sitios de lanzamiento en un mapa
* **TAREA 2:** Marcar los lanzamientos exitosos/fallidos de cada sitio en el mapa
* **TAREA 3:** Calcular las distancias entre un sitio de lanzamiento y sus proximidades

Después de completar las tareas anteriores, debería poder encontrar algunos patrones geográficos sobre los sitios de lanzamiento.

In [14]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
import requests
import io

In [3]:
## Tarea 1: Marcar todos los sitios de lanzamiento en un mapa

Primero, intentemos agregar la ubicación de cada sitio en un mapa usando las coordenadas de latitud y longitud del sitio.

El siguiente conjunto de datos con el nombre spacex_launch_geo.csv es un conjunto de datos ampliado con latitud y longitud agregadas para cada sitio.

In [15]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response = requests.get(URL)

if response.status_code == 200:
    spacex_csv_file = response.content
    spacex_df = pd.read_csv(io.BytesIO(spacex_csv_file))
    print(spacex_df.head())
else:
    print(f"Error al descargar el archivo: {response.status_code}")

   Flight Number        Date Time (UTC) Booster Version  Launch Site  \
0              1  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1              2  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2              3  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3              4  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4              5  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  Payload Mass (kg)  \
0               Dragon Spacecraft Qualification Unit                0.0   
1  Dragon demo flight C1, two CubeSats,  barrel o...                0.0   
2                             Dragon demo flight C2+              525.0   
3                                       SpaceX CRS-1              500.0   
4                                       SpaceX CRS-2              677.0   

       Orbit         Customer        Landing Outcome  class        Lat  \
0        LEO           SpaceX  Failure   (

Ahora, puedes echar un vistazo a cuáles son las coordenadas de cada sitio.

In [7]:
# Seleccione subcolumnas relevantes: `Sitio de lanzamiento`, `Lat(Latitud)`, `Long(Longitud)`, `clase`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Las coordenadas anteriores son simplemente números simples que no pueden brindarle ninguna información intuitiva sobre dónde están esos sitios de lanzamiento. Si es muy bueno en geografía, puede interpretar esos números directamente en su mente. Si no, eso también está bien. Visualicemos esas ubicaciones marcándolas en un mapa.

Primero debemos crear un objeto de mapa Folium, con una ubicación central inicial que será el Centro Espacial Johnson de la NASA en Houston, Texas.

In [9]:
# La ubicación de inicio es el Centro Espacial Johnson de la NASA
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

Podríamos usar folium.Circle para agregar un área de círculo resaltada con una etiqueta de texto en una coordenada específica. Por ejemplo,

In [11]:
# Crea un círculo azul en las coordenadas del Centro Espacial Johnson de la NASA con una etiqueta emergente que muestra su nombre
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Crea un círculo azul en las coordenadas del Centro Espacial Johnson de la NASA con un ícono que muestra su nombre
marker = folium.map.Marker(
    nasa_coordinate,
    # Crear un icono como etiqueta de texto
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

y deberías encontrar un pequeño círculo amarillo cerca de la ciudad de Houston y puedes hacer zoom para ver un círculo más grande.

Ahora, agreguemos un círculo para cada sitio de lanzamiento en el marco de datos launch_sites

TODO: Crear y agregar folium.Circle y folium.Marker para cada sitio de lanzamiento en el mapa del sitio

Un ejemplo de folium.Circulo:

`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`

Un ejemplo de folio.Marker:

`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`

In [16]:
# Iterar sobre cada fila en el marco de datos launch_sites_df
for index, row in launch_sites_df.iterrows():
    # Extraer la latitud y la longitud de cada sitio de lanzamiento
    lat = row['Lat']
    lon = row['Long']
    launch_site = row['Launch Site']
    
    # Crear un círculo en las coordenadas del sitio de lanzamiento con una etiqueta emergente que muestra su nombre
    circle = folium.Circle([lat, lon], radius=1000, color='#3186cc', fill=True).add_child(folium.Popup(launch_site))
    
    # Crear un marcador en las coordenadas del sitio de lanzamiento con un icono que muestra su nombre
    marker = folium.map.Marker(
        [lat, lon],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#3186cc;"><b>%s</b></div>' % launch_site,
            )
        )
    
    # Agregar el círculo y el marcador al mapa
    site_map.add_child(circle)
    site_map.add_child(marker)

# Mostrar el mapa con los círculos y marcadores de los sitios de lanzamiento
site_map


El mapa generado con los sitios de lanzamiento marcados debería verse similar al siguiente:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>

Ahora, puedes explorar el mapa acercando o alejando las áreas marcadas
e intentar responder las siguientes preguntas:

* ¿Están todos los sitios de lanzamiento cerca de la línea del Ecuador?
* ¿Están todos los sitios de lanzamiento muy cerca de la costa?

Intenta también explicar tus hallazgos.

In [17]:
# Tarea 2: Marcar los lanzamientos exitosos/fallidos de cada sitio en el mapa

A continuación, intentemos mejorar el mapa agregando los resultados de lanzamiento de cada sitio y veamos qué sitios tienen tasas de éxito altas. Recuerde que el marco de datos spacex_df tiene registros de lanzamiento detallados y la columna de clase indica si este lanzamiento fue exitoso o no.

In [18]:
spacex_df.tail(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
46,43,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200.00,GTO,SES EchoStar,Success (drone ship),1,28.573255,-80.646895
47,44,2017-10-30,19:34:00,F9 B4 B1042.1,KSC LC-39A,Koreasat 5A,3500.00,GTO,KT Corporation,Success (drone ship),1,28.573255,-80.646895
48,54,2018-05-11,20:14:00,F9 B5 B1046.1,KSC LC-39A,Bangabandhu-1,3600.00,GTO,Thales-Alenia/BTRC,Success (drone ship),1,28.573255,-80.646895
49,45,2017-12-15,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205.00,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820
50,47,2018-01-08,1:00:00,F9 B4 B1043.1,CCAFS SLC-40,Zuma,3696.65,LEO,Northrop Grumman,Success (ground pad),1,28.563197,-80.576820
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.00,GTO,SES,Controlled (ocean),0,28.563197,-80.576820
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.00,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.576820
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.00,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.576820
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.00,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.576820
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.00,GTO,SES,No attempt,0,28.563197,-80.576820


A continuación, crearemos marcadores para todos los registros de lanzamiento. Si un lanzamiento fue exitoso `(class=1)`, entonces usamos un marcador verde y si un lanzamiento falló, usamos un marcador rojo `(class=0)`

Tenga en cuenta que un lanzamiento solo se produce en uno de los cuatro sitios de lanzamiento, lo que significa que muchos registros de lanzamiento tendrán exactamente la misma coordenada. Los grupos de marcadores pueden ser una buena forma de simplificar un mapa que contiene muchos marcadores que tienen la misma coordenada.

Primero, creemos un objeto `MarkerCluster`

In [19]:
marker_cluster = MarkerCluster()

*TODO:* Crear una nueva columna en el marco de datos `spacex_df` llamada `marker_color` para almacenar los colores de los marcadores según el valor de `class`

In [20]:
# Aplicar una función para verificar el valor de la columna `class`
# Si class=1, el valor de marker_color será verde
# Si class=0, el valor de marker_color será rojo

In [21]:
# Definir una función para determinar el color del marcador
def get_marker_color(launch_class):
    if launch_class == 1:
        return 'green'
    else:
        return 'red'

# Iterar sobre cada fila en el marco de datos launch_sites_df
for index, row in spacex_df.iterrows():
    # Extraer la latitud, longitud, nombre del sitio de lanzamiento y clase
    lat = row['Lat']
    lon = row['Long']
    launch_site = row['Launch Site']
    launch_class = row['class']
    
    # Obtener el color del marcador basado en el valor de la clase
    marker_color = get_marker_color(launch_class)
    
    # Crear un círculo en las coordenadas del sitio de lanzamiento con una etiqueta emergente que muestra su nombre
    circle = folium.Circle([lat, lon], radius=1000, color=marker_color, fill=True).add_child(folium.Popup(launch_site))
    
    # Crear un marcador en las coordenadas del sitio de lanzamiento con un icono que muestra su nombre
    marker = folium.map.Marker(
        [lat, lon],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:{marker_color};"><b>{launch_site}</b></div>',
            )
        )
    
    # Agregar el círculo y el marcador al mapa
    site_map.add_child(circle)
    site_map.add_child(marker)

# Mostrar el mapa con los círculos y marcadores de los sitios de lanzamiento
site_map


*TODO:* Para cada resultado de lanzamiento en el marco de datos `spacex_df`, agregue un `folium.Marker` a `marker_cluster`

A partir de los marcadores etiquetados por color en los grupos de marcadores, debería poder identificar fácilmente qué sitios de lanzamiento tienen tasas de éxito relativamente altas.

In [22]:
# TAREA 3: Calcular las distancias entre un sitio de lanzamiento y sus proximidades

A continuación, debemos explorar y analizar la proximidad de los sitios de lanzamiento.

Primero, agreguemos una `Posición del mouse` en el mapa para obtener las coordenadas del mouse sobre un punto del mapa. De esta manera, mientras explora el mapa, puede encontrar fácilmente las coordenadas de cualquier punto de interés (como el ferrocarril).

In [23]:
# Agregue la posición del mouse para obtener las coordenadas (latitud, longitud) al pasar el mouse sobre el mapa
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Ahora acérquese a un sitio de lanzamiento y explore su proximidad para ver si puede encontrar fácilmente algún ferrocarril, autopista, costa, etc. Mueva el mouse a estos puntos y marque sus coordenadas (mostradas en la parte superior izquierda) para indicar la distancia al sitio de lanzamiento.

In [26]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # Radio aproximado de la Tierra en km.
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

TODO: Marque un punto en la costa más cercana usando MousePosition y calcule la distancia entre el punto de la costa y el sitio de lanzamiento.

In [25]:
# encuentra las coordenadas de la línea de costa más cercana
# p. ej.: Lat: 28.56367 Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [28]:
# Coordenadas del sitio de lanzamiento (ejemplo)
launch_site_coords = [20.6597, -103.3496]

# Coordenadas del punto en la costa más cercana (ejemplo)
coast_point_coords = [20.6743, -103.405]

# Agregar un marcador en el sitio de lanzamiento
folium.Marker(location=launch_site_coords, popup="Sitio de Lanzamiento").add_to(site_map)

# Agregar un marcador en el punto de la costa más cercana
folium.Marker(location=coast_point_coords, popup="Punto en la Costa Más Cercana").add_to(site_map)

# Calcular la distancia entre el sitio de lanzamiento y el punto en la costa
distance = calculate_distance(launch_site_coords[0], launch_site_coords[1], coast_point_coords[0], coast_point_coords[1])

# Mostrar la distancia
print("Distancia entre el sitio de lanzamiento y el punto en la costa más cercana:", distance, "km")

# Mostrar el mapa
site_map

Distancia entre el sitio de lanzamiento y el punto en la costa más cercana: 5.989923922320426 km


In [ ]:
# Cree y agregue un foluim.Marker en el punto de costa más cercano seleccionado en el mapa
# Muestre la distancia entre el punto de costa y el sitio de lanzamiento usando la propiedad del ícono
# por ejemplo
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

In [31]:
coordinate = [20.6743, -103.405]

distance_marker = folium.Marker(
    coordinate,
    icon=DivIcon(
       icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
        )
    )

site_map.add_child(circle)
site_map.add_child(marker)

site_map

TODO: Dibujar una polilínea entre un sitio de lanzamiento y el punto de la costa seleccionado

In [33]:
# Crea un objeto `folium.PolyLine` usando las coordenadas de la línea de costa y las coordenadas del sitio de lanzamiento
coordinates = [launch_site_coords, coast_point_coords]
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

TODO: De manera similar, puedes dibujar una línea entre un sitio de lanzamiento y su ciudad, ferrocarril, autopista, etc. más cercana. Primero debes usar la posición del mouse para encontrar sus coordenadas en el mapa.

In [ ]:
# Crea un marcador con la distancia a la ciudad, vía férrea, autopista, etc. más cercana.
# Dibuja una línea entre el marcador y el sitio de lanzamiento

Después de trazar líneas de distancia a las proximidades, puede responder fácilmente las siguientes preguntas:

* ¿Los sitios de lanzamiento están cerca de las vías férreas?
* ¿Los sitios de lanzamiento están cerca de las autopistas?
* ¿Los sitios de lanzamiento están cerca de la costa?
* ¿Los sitios de lanzamiento se mantienen a cierta distancia de las ciudades?

Intente también explicar sus hallazgos.

## Próximos pasos:
Ahora ha descubierto muchos datos interesantes relacionados con la ubicación de los sitios de lanzamiento utilizando Folium, de una manera muy interactiva. A continuación, deberá crear un panel de control utilizando Ploty Dash con registros de lanzamiento detallados.